### 第一步：从"市场"导入预置的数据集
目前ModelArts SDK/API暂时不支持对"市场"的操作，在本地使用OBS SDK上传数据；或使用web console导入数据
(ModelArts notebook中无法访问网络，只能通过web console操作导入数据)

In [72]:
## 准备SDK
!pip install esdk-obs-python

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple


In [73]:
%env USER_ACCESS_KEY=my_ak
%env USER_SECRET_ACCESS_KEY=my_sk
%env PROJECT_ID=my_projid

env: USER_ACCESS_KEY=SC36F1SPFVDDCD8MDFNG
env: USER_SECRET_ACCESS_KEY=V5MEsWv9ws8aVEgepEaflE01Tow5ybdRZUoZagIp
env: PROJECT_ID=cf1bed56cdc045b196278cace96852a8


In [74]:
import os
from obs import ObsClient

AK=os.getenv('USER_ACCESS_KEY')
SK=os.getenv('USER_SECRET_ACCESS_KEY')
PROJECT_ID=os.getenv('PROJECT_ID')

obs_endpoint=os.getenv('MINER_OBS_URL')
if obs_endpoint is None:
    obs_endpoint='obs.cn-north-1.myhuaweicloud.com'


obsClient = ObsClient(
    access_key_id=AK,
    secret_access_key=SK,
    server=obs_endpoint
)

In [75]:
TARGET_BUCKET = 'ma-workflow'
resp = obsClient.headBucket(TARGET_BUCKET)
if resp.status == 404 and obsClient.createBucket(TARGET_BUCKET).status >= 300:
    print("Failed to create bucket{}".format(TARGET_BUCKET))


In [77]:
resp = obsClient.putFile(TARGET_BUCKET, 'mnist/code/train_mnist.py', file_path='./train_mnist.py')
print(resp.body)
if resp.status >= 300:
    print('Failed to put file')  

{'versionId': 'G0011168E633E04EFFFF804A0F4C5576', 'etag': '"ecaf01772735cc57e70172da36c2868b"', 'objectUrl': 'https://ma-workflow.obs.cn-north-1.myhuaweicloud.com/mnist/code/train_mnist.py'}


创建ModelArts客户端，注意需要在HOME目录下配置.modelarts/config.json
详情参见[初始化ModelArts SDK](https://support.huaweicloud.com/sdkreference-modelarts/modelarts_04_0006.html)
注意：非企业用户account为空，只填username

In [78]:
from modelarts import config
from modelarts.client.api import *
from modelarts.client.rest import ApiException
maClient = config.create_client(context='default')

/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


创建训练作业
- 参考ModelArts SDK说明，body与本目录下training_job.json一致
- 对于文档中参数说明不明确之处，在web console中创建训练任务并观察请求参数，request body保存在sample_training_job.json中
- spec_id的获取参考文档 [查看作业资源规格](https://support.huaweicloud.com/sdkreference-modelarts/modelarts_04_0074.html)
- engine_id的获取参考文档 [查看作业引擎规格](https://support.huaweicloud.com/sdkreference-modelarts/modelarts_04_0075.html)


In [85]:
# 注意事项xx_url应该是目录，必须以/结尾
# engine_id=29: MXNet-1.2.1-python3.6
# model_id优先级高于app_url & boot_file_url，若指定model_id则优先从已有模型中读取参数并进行训练

training_job_body = {
  "job_name": "mxnet-mnist",
  "job_desc": "mxnet-mnist with ModelArts SDK",
  "config": {
    "worker_server_num": 1,
#     "model_id": 3,
    "app_url": "/ma-workflow/mnist/code/",
    "boot_file_url": "/ma-workflow/mnist/code/train_mnist.py",
    "data_source": [
      {
        "type": "obs",
        "data_url": "/ma-workflow/mnist/"
      }
    ],
    "train_url": "/ma-workflow/mnist/output/",
    "log_url": "/ma-workflow/mnist/log/",
    "spec_id": 2,
    "engine_id": 29
  }
}

trainJobApi = TrainJobApi(maClient)
try:
    training_job = trainJobApi.create_training_job(project_id=PROJECT_ID, body=training_job_body)
    job_id = training_job.job_id
    version_id = training_job.version_id
except ApiException as e:
    resp_obj = json.loads(e.body)
    print(e)
    if resp_obj['error_code'] == 'ModelArts.0103': # 重名，作业已存在，创建作业版本
        # 当前SDK需要先列出job list，再按照job_name查找到job_id，才能创建作业版本
        # 删除job也需要job_id
        print("Let's createa job version, it's a pretty tough task for now, we'll do it later!")
        
    
        

/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


监听训练任务的状态

In [86]:
# info = trainJobApi.view_training_job(project_id=PROJECT_ID, job_id=job_id, version_id=version_id)
# print(info)
from time import sleep
elapsed_seconds = 0
TEN_MINUTES = 60 * 10  # Make it longer for computation consuming training jobs
current_status = 0
while True:
    info = trainJobApi.view_training_job(project_id=PROJECT_ID, job_id=job_id, version_id=version_id)

    if current_status == info.status:
        continue
    else:
        current_status = info.status
    
    print(info.status)
    if info.status == 1:
        print('Job {}@{} initializing'.format(job_id, version_id))  
        
    elif info.status == 8:
        print('Job {}@{} running'.format(job_id, version_id))
    elif info.status == 10:
        print('Job {}@{} complete'.format(job_id, version_id))
        break
    elif info.status == 11:
        print('Job {}@{} failed'.format(job_id, version_id))
        print(info)
        break
        
    elapsed_seconds += 5
    if elapsed_seconds >= TEN_MINUTES:
        print('Job {}@{} timeout'.format(job_id, version_id))
        break
            

/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


1
Job 19638@28827 initializing


/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning

8
Job 19638@28827 running


/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning

/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning

10
Job 19638@28827 complete


导入模型，上传config.json和customize_service.py到model目录下

In [88]:
resp = obsClient.putFile(TARGET_BUCKET, 'mnist/output/model/config.json', file_path='./config.json')
print('upload config.json for model:\n{}\n'.format(resp.body))
if resp.status >= 300:
    print('Failed to put file')  

resp = obsClient.putFile(TARGET_BUCKET, 'mnist/output/model/customize_service.py', file_path='./customize_service.py')
print('upload customize_service.py for model:\n{}\n'.format(resp.body))
if resp.status >= 300:
    print('Failed to put file')  
    

upload config.json for model:
{'versionId': 'G0011168E64529C1FFFF80510FC48FD3', 'etag': '"bd6a5ac6acf304030075497d575669e1"', 'objectUrl': 'https://ma-workflow.obs.cn-north-1.myhuaweicloud.com/mnist/output/model/config.json'}

upload customize_service.py for model:
{'versionId': 'G0011168E64529F3FFFF80510FC48FD4', 'etag': '"f242ba3a83d8127807ca7b2cdfd0fbcb"', 'objectUrl': 'https://ma-workflow.obs.cn-north-1.myhuaweicloud.com/mnist/output/model/customize_service.py'}



In [92]:
modelApi = ModelApi(maClient)

model_body = {
  "model_name": "mxnet-mnist-model",
  "model_version": "0.0.1",
#   source_location到model目录上一层即可
  "source_location": "https://ma-workflow.obs.cn-north-1.myhwclouds.com/mnist/output",
#   注释掉的参数为必填，但是从配置文件中读取
  "model_type": "MXNet",
  "model_algorithm": "image_classification",
#   "input_params": [
#     {
#       "url": "/object_detection",
#       "param_name": "field0",
#       "param_type": "int"
#     }
#   ],
#   "output_params": [
#     {
#       "url": "/object_detection",
#       "param_name": "res0",
#       "param_type": "int"
#     }
#   ],
#   "dependencies": [
#     {
#       "installer": "pip",
#       "packages": [
#         {
#           "package_name": "numpy",
#           "package_version": "1.5.0",
#           "restraint": "ATLEAST"
#         }
#       ]
#     }
#   ]
}

imported_model = modelApi.create_the_model(project_id=PROJECT_ID, body=model_body)

/home/lance/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [93]:
print(imported_model)

{'model_id': '8319ccbf-408f-494c-8925-93866727f6c9'}


In [67]:
serviceApi = ServiceApi(maClient)

body = {
    "service_name": "mxnet-mnist-service",
    "description": "mxnet mnist service",
    "infer_type": "real-time",
    "config": [
        {
            "model_id": "22d7a217-86d1-4ef1-94cf-5661de31c027",
            "weight": "70",
            "specification": "c2.m8.g1",
            "instance_count": 1,
            "envs": {
                "model_name": "demo-yb-8606V001",
                "load_epoch": "0"
            }
        },
        {
            "model_id": "22d7a217-86d1-4ef1-94cf-5661de31c025",
            "weight": "30",
            "specification": "c2.m8.g1",
            "instance_count": 1
        }
    ]
}
serviceApi.create_service(project_id=PROJECT_ID, body = body)